In [15]:
import pandas as pd
import pickle
import spacy
from tqdm import tqdm

In [2]:
df = pd.read_json("/Users/justinvhuang/Desktop/CSE-6242-Group-Project/web_scraping/fin_anime_df.json")

In [3]:
# Specify the path to your pickle file
pickle_file_path = "/Users/justinvhuang/Desktop/CSE-6242-Group-Project/web_scraping/popular_dict_10.pkl"

# Open the pickle file in read-binary mode
with open(pickle_file_path, "rb") as f:
    # Load the object from the pickle file
    loaded_object = pickle.load(f)

In [4]:
df2 = pd.read_csv("/Users/justinvhuang/Desktop/CSE-6242-Group-Project/anime-dataset-2023.csv")

In [5]:
pop_list = df2[df2['anime_id'].isin(pd.DataFrame(loaded_object)['anime_id'].tolist())].sort_values('Favorites', ascending = False).head(10)['anime_id'].tolist()

In [6]:
df['popular_recs'] = [pop_list] * len(df)

In [7]:
# Specify the path to your pickle file
pickle_file_path = "/Users/justinvhuang/Desktop/CSE-6242-Group-Project/web_scraping/anime_recommendations_item_knn_CF_10k_num.pkl"

# Open the pickle file in read-binary mode
with open(pickle_file_path, "rb") as f:
    # Load the object from the pickle file
    loaded_object_cf = pickle.load(f)

In [8]:
anime_anime_id_series = pd.Series(loaded_object_cf)


In [9]:
df['cf_recs'] = df['anime_anime_id'].map(anime_anime_id_series)

In [10]:
pd.set_option('display.max_columns', None)
df

,imdb_title_title,imdb_title_titleId,anime_Rating,anime_English name,anime_Genres,anime_Other name,anime_Score,anime_Synopsis,anime_anime_id,imdb_name_basics_primaryName,plot,keywords,image,Producers,Licensors,Studios,Image URL,Episodes,Genres,Source,Favorites,Aired,Members,Duration,text,Name,anime_id,popular_recs,cf_recs
0,UNKNOWN,UNKNOWN,PG-13 - Teens 13 or older,UNKNOWN,"Action, Drama",頭文字〈イニシャル〉D FOURTH STAGE,8.16,Takumi Fujiwara finally joins Ryousuke and Kei...,18,UNKNOWN,"Initial D (Japanese: 頭文字イニシャル D, Hepburn: Inis...",UNKNOWN,UNKNOWN,"OB Planning, Studio Jack",Funimation,A.C.G.T.,https://cdn.myanimelist.net/images/anime/9/105...,24.0,"Action, Drama",Manga,1237,"Apr 17, 2004 to Feb 18, 2006",173710,27 min per ep,"Initial D (Japanese: 頭文字イニシャル D, Hepburn: Inis...",Initial D Fourth Stage,18,"[21, 1575, 1, 227, 431, 121, 226, 263, 2025, 47]","[2608, 263, 245, 5258, 264, 265, 135, 71, 627,..."
1,UNKNOWN,UNKNOWN,PG - Children,UNKNOWN,"Action, Sports",リングにかけろ１,6.39,"In order to fulfill their dead father's wish, ...",23,UNKNOWN,"Ring ni Kakero (Japanese: リングにかけろ, Hepburn: Ri...",UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,Toei Animation,https://cdn.myanimelist.net/images/anime/1146/...,12.0,"Action, Sports",Manga,6,"Oct 6, 2004 to Dec 15, 2004",4581,25 min per ep,"Ring ni Kakero (Japanese: リングにかけろ, Hepburn: Ri...",Ring ni Kakero 1,23,"[21, 1575, 1, 227, 431, 121, 226, 263, 2025, 47]","[48374, 51190, 9949, 9947, 20077, 16804, 30034..."
2,UNKNOWN,UNKNOWN,PG-13 - Teens 13 or older,D.C.~Da Capo~,"Drama, Romance",D.C.～ダ・カーポ～,6.72,Hatsunejima abounds in mysteries—one of which ...,62,UNKNOWN,"Da Capo (〜ダ・カーポ〜, Da Kāpo, commonly abbreviate...",UNKNOWN,UNKNOWN,"Lantis, King Records",UNKNOWN,"feel., Zexcs",https://cdn.myanimelist.net/images/anime/4/739...,26.0,"Drama, Romance",Visual novel,194,"Jul 5, 2003 to Dec 27, 2003",74844,24 min per ep,"Da Capo (〜ダ・カーポ〜, Da Kāpo, commonly abbreviate...",D.C.: Da Capo,62,"[21, 1575, 1, 227, 431, 121, 226, 263, 2025, 47]","[291, 2595, 3627, 5216, 4744, 2494, 3299, 119,..."
3,UNKNOWN,UNKNOWN,R+ - Mild Nudity,DearS,"Comedy, Romance, Sci-Fi, Ecchi",ディアーズ,6.56,"One year ago, a UFO containing 150 aliens cras...",63,UNKNOWN,Bears are carnivoran mammals of the family Urs...,UNKNOWN,UNKNOWN,"Bandai Visual, Hal Film Maker, Genco, Lantis","Discotek Media, Geneon Entertainment USA",Daume,https://cdn.myanimelist.net/images/anime/1207/...,12.0,"Comedy, Romance, Sci-Fi, Ecchi",Manga,346,"Jul 11, 2004 to Sep 26, 2004",145533,23 min per ep,Bears are carnivoran mammals of the family Urs...,DearS,63,"[21, 1575, 1, 227, 431, 121, 226, 263, 2025, 47]","[241, 59, 487, 2494, 133, 3503, 53, 50, 4999, ..."
4,UNKNOWN,UNKNOWN,PG-13 - Teens 13 or older,UNKNOWN,"Comedy, Drama, Romance",愛してるぜ ベイベ★★,7.44,Katakura Kippei is in every way a high school ...,102,UNKNOWN,"Aishiteruze Baby (愛してるぜ ベイベ★★, Aishiteruze Bei...",UNKNOWN,UNKNOWN,"Animax, Toei Video, Sunny Side Up",UNKNOWN,TMS Entertainment,https://cdn.myanimelist.net/images/anime/2/696...,26.0,"Comedy, Drama, Romance",Manga,546,"Apr 3, 2004 to Oct 9, 2004",101134,25 min per ep,"Aishiteruze Baby (愛してるぜ ベイベ★★, Aishiteruze Bei...",Aishiteruze Baby★★,102,"[21, 1575, 1, 227, 431, 121, 226, 263, 2025, 47]","[1222, 3731, 53, 120, 2034, 394, 61, 101, 3470..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11560,I've Always Liked You,tt5889338,UNKNOWN,I've Always Liked You,Romance,ずっと前から好きでした。～告白実行委員会～,7.22,Love is blooming at Sakuragaoka High School. N...,31245,"Kana Asumi, Gero, Yoshimi Narita, Momo Asakura...",Zutto Mae Kara Suki Deshita: Kokuhaku Jikkō Ii...,"love,anime animation",https://m.media-amazon.com/images/M/MV5BOTcxNz...,"Aniplex, Dentsu, Kadokawa Shoten, Movic, Fuji ...",Aniplex of America,Qualia Animation,https://cdn.myanimelist.net/images/anime/3/821...,1.0,Romance,Music,632,"Apr 23, 2016",220711,1 hr 3 min,Zutto Mae Kara Suki Deshita: Kokuhaku Jikkō Ii...,Zutto Mae kara Suki deshita. Kokuhaku Jikkou I...

In [21]:
df['tokens'] = df['text'] + " " + df['Genres']  + " " + df['Source']  + " " + df['keywords'] + " " + df['Producers'] + " " + df['imdb_name_basics_primaryName'] + " " + df['anime_Synopsis'] + " " + df['Studios'] + " " + df['Name']

In [22]:
# Load the English language model in spaCy
nlp = spacy.load("en_core_web_sm")

# Define the preprocessing functions
def lemmatize_text(text):
    doc = nlp(text)
    return [token.lemma_ for token in doc]

def remove_stopwords(tokens):
    return [token for token in tokens if not nlp.vocab[token].is_stop]

def remove_punctuations(tokens):
    return [token for token in tokens if not nlp.vocab[token].is_punct]

# Preprocess the text column in the DataFrame
def preprocess_dataframe(df, column_name):
    tqdm.pandas()
    
    # Lowercase text column
    df[column_name] = df[column_name].str.lower()
    
    # Lemmatize text column
    df[column_name] = df[column_name].progress_apply(lemmatize_text)
    
    # Remove stopwords from text column
    df[column_name] = df[column_name].progress_apply(remove_stopwords)
    
    # Remove punctuations from text column
    df[column_name] = df[column_name].progress_apply(remove_punctuations)
    
    return df


In [23]:
fin_df = preprocess_dataframe(df, 'tokens')

100%|██████████| 11565/11565 [00:01<00:00, 6621.69it/s]


In [24]:
fin_df

,imdb_title_title,imdb_title_titleId,anime_Rating,anime_English name,anime_Genres,anime_Other name,anime_Score,anime_Synopsis,anime_anime_id,imdb_name_basics_primaryName,plot,keywords,image,Producers,Licensors,Studios,Image URL,Episodes,Genres,Source,Favorites,Aired,Members,Duration,text,Name,anime_id,popular_recs,cf_recs,tokens
0,UNKNOWN,UNKNOWN,PG-13 - Teens 13 or older,UNKNOWN,"Action, Drama",頭文字〈イニシャル〉D FOURTH STAGE,8.16,Takumi Fujiwara finally joins Ryousuke and Kei...,18,UNKNOWN,"Initial D (Japanese: 頭文字イニシャル D, Hepburn: Inis...",UNKNOWN,UNKNOWN,"OB Planning, Studio Jack",Funimation,A.C.G.T.,https://cdn.myanimelist.net/images/anime/9/105...,24.0,"Action, Drama",Manga,1237,"Apr 17, 2004 to Feb 18, 2006",173710,27 min per ep,"Initial D (Japanese: 頭文字イニシャル D, Hepburn: Inis...",Initial D Fourth Stage,18,"[21, 1575, 1, 227, 431, 121, 226, 263, 2025, 47]","[2608, 263, 245, 5258, 264, 265, 135, 71, 627,...","[initial, d, japanese, 頭文字イニシャル, d, hepburn, i..."
1,UNKNOWN,UNKNOWN,PG - Children,UNKNOWN,"Action, Sports",リングにかけろ１,6.39,"In order to fulfill their dead father's wish, ...",23,UNKNOWN,"Ring ni Kakero (Japanese: リングにかけろ, Hepburn: Ri...",UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,Toei Animation,https://cdn.myanimelist.net/images/anime/1146/...,12.0,"Action, Sports",Manga,6,"Oct 6, 2004 to Dec 15, 2004",4581,25 min per ep,"Ring ni Kakero (Japanese: リングにかけろ, Hepburn: Ri...",Ring ni Kakero 1,23,"[21, 1575, 1, 227, 431, 121, 226, 263, 2025, 47]","[48374, 51190, 9949, 9947, 20077, 16804, 30034...","[ring, ni, kakero, japanese, リングにかけろ, hepburn,..."
2,UNKNOWN,UNKNOWN,PG-13 - Teens 13 or older,D.C.~Da Capo~,"Drama, Romance",D.C.～ダ・カーポ～,6.72,Hatsunejima abounds in mysteries—one of which ...,62,UNKNOWN,"Da Capo (〜ダ・カーポ〜, Da Kāpo, commonly abbreviate...",UNKNOWN,UNKNOWN,"Lantis, King Records",UNKNOWN,"feel., Zexcs",https://cdn.myanimelist.net/images/anime/4/739...,26.0,"Drama, Romance",Visual novel,194,"Jul 5, 2003 to Dec 27, 2003",74844,24 min per ep,"Da Capo (〜ダ・カーポ〜, Da Kāpo, commonly abbreviate...",D.C.: Da Capo,62,"[21, 1575, 1, 227, 431, 121, 226, 263, 2025, 47]","[291, 2595, 3627, 5216, 4744, 2494, 3299, 119,...","[da, capo, 〜ダ・カーポ〜, da, kāpo, commonly, abbrev..."
3,UNKNOWN,UNKNOWN,R+ - Mild Nudity,DearS,"Comedy, Romance, Sci-Fi, Ecchi",ディアーズ,6.56,"One year ago, a UFO containing 150 aliens cras...",63,UNKNOWN,Bears are carnivoran mammals of the family Urs...,UNKNOWN,UNKNOWN,"Bandai Visual, Hal Film Maker, Genco, Lantis","Discotek Media, Geneon Entertainment USA",Daume,https://cdn.myanimelist.net/images/anime/1207/...,12.0,"Comedy, Romance, Sci-Fi, Ecchi",Manga,346,"Jul 11, 2004 to Sep 26, 2004",145533,23 min per ep,Bears are carnivoran mammals of the family Urs...,DearS,63,"[21, 1575, 1, 227, 431, 121, 226, 263, 2025, 47]","[241, 59, 487, 2494, 133, 3503, 53, 50, 4999, ...","[bear, carnivoran, mammal, family, ursidae, cl..."
4,UNKNOWN,UNKNOWN,PG-13 - Teens 13 or older,UNKNOWN,"Comedy, Drama, Romance",愛してるぜ ベイベ★★,7.44,Katakura Kippei is in every way a high school ...,102,UNKNOWN,"Aishiteruze Baby (愛してるぜ ベイベ★★, Aishiteruze Bei...",UNKNOWN,UNKNOWN,"Animax, Toei Video, Sunny Side Up",UNKNOWN,TMS Entertainment,https://cdn.myanimelist.net/images/anime/2/696...,26.0,"Comedy, Drama, Romance",Manga,546,"Apr 3, 2004 to Oct 9, 2004",101134,25 min per ep,"Aishiteruze Baby (愛してるぜ ベイベ★★, Aishiteruze Bei...",Aishiteruze Baby★★,102,"[21, 1575, 1, 227, 431, 121, 226, 263, 2025, 47]","[1222, 3731, 53, 120, 2034, 394, 61, 101, 3470...","[aishiteruze, baby, 愛してるぜ, ベイベ, ★, ★, aishiter..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11560,I've Always Liked You,tt5889338,UNKNOWN,I've Always Liked You,Romance,ずっと前から好きでした。～告白実行委員会～,7.22,Love is blooming at Sakuragaoka High School. N...,31245,"Kana Asumi, Gero, Yoshimi Narita, Momo Asakura...",Zutto Mae Kara Suki Deshita: Kokuhaku Jikkō Ii...,"love,anime animation",https://m.media-amazon.com/images/M/MV5BOTcxNz...,"Aniplex, Dentsu, Kad